In [ ]:
%matplotlib inline

In [ ]:
from enum import Enum
from tqdm.notebook import tqdm
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import ContinuousSpace
from mesa.batchrunner import BatchRunner
from mesa.visualization.modules import CanvasGrid
from mesa.datacollection import DataCollector
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Line
from bokeh.models.annotations import Legend
from bokeh.palettes import Category10
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex
import pandas as pd
import panel as pn
import math
import itertools
pn.extension()

# Mesa
> Mesa is a modular framework for building, analyzing and visualizing **agent-based models**.

[Documentation](https://mesa.readthedocs.io/en/master/overview.html)

# Introduction

The simulation is composed of the following components:
- `Model` class - representing the model.
- `Agent` class - representing the agent.
- `Scheduler` class - agent activation strategy
- `step` method - simulation logic

In the simulation we will have a continouous space. Agents will move within it and infect other agents.

# Simulation model

In [ ]:
class State(Enum):
    UNAFFECTED = 0
    INFECTED = 1
    RECOVERED = 2
    DEAD = 3

### _Ex. 1. Warmup_

In [ ]:
def change_position(pos, speed, heading):
    x, y = pos
    return # TODO 1: return new position

## Agent

### _Ex. 2. Agent implementation_

In [ ]:
class EpidemicAgent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model, state):
        super().__init__(unique_id, model)
        self.state = state
        self.infection_probability = model.infection_p
        self.infection_radius = model.infection_r
        self.speed = model.initial_speed
        self.heading = self.random.uniform(0, 2 * math.pi)
        self.time_infected_min = model.infection_time_min
        self.time_infected = self.random.uniform(0, self.time_infected_min) if state == State.INFECTED else 0
        self.infection_end_p = model.infection_end_p
        self.death_p = model.death_p

    def move(self):
        # Move agent
        new_pos = change_position(self.pos, self.speed, self.heading)
        self.model.grid.move_agent(self, new_pos)
        # Update velocity
        self.speed += self.random.uniform(-3, 3)
        self.heading += self.random.uniform(-math.pi/4, math.pi/4)
    
    def infect_others(self):
        # TODO 2: Select agents within `infection_radius`
        # TODO 2: Infect others with `infection_probability`
        pass

    def become_infected(self):
        if self.state == State.UNAFFECTED:
            self.state = State.INFECTED
            self.time_infected = 0
    
    def try_die_or_recover(self):
        # TODO 6: implement recovery or death after no sooner then `time_infected_min`
        # TIP: use the following fields: `time_infected`, `time_infected_min`, `solution_p`, `death_p`
        # TIP: use `die` and `recover` methods
        pass
    
    def die(self):
        self.model.grid.remove_agent(self)
        self.model.schedule.remove(self)
        self.state = State.DEAD

    def recover(self):
        self.state = State.RECOVERED

    def step(self):
        # TODO 2: Move agent
        
        if self.state == State.INFECTED:
            self.time_infected += 1
            self.infect_others()
            self.try_die_or_recover()

In [ ]:
def compute_alive(model):
    return len(model.schedule.agents)

In [ ]:
def compute_infected(model):
    return 0 # TODO 5: return number of sick agents

In [ ]:
def compute_dead(model):
    return model.num_agents - compute_alive(model)

In [ ]:
def compute_recovered(model):
    return len([a for a in model.schedule.agents if a.state == State.RECOVERED])

## Model

### Ex. 3. Model implementation

In [ ]:
class EpidemicModel(Model):
    """A model with some number of agents."""
    def __init__(self,
                 N,
                 initial_infection_p,
                 infection_p,
                 infection_r,
                 infection_time_min,
                 infection_end_p,
                 death_p,
                 initial_speed):
        x_max, y_max = 1000, 1000
        self.num_agents = N
        
        self.initial_infection_p = initial_infection_p
        self.infection_p = infection_p
        self.infection_r = infection_r
        self.infection_time_min = infection_time_min
        self.infection_end_p = infection_end_p
        self.death_p = death_p
        self.initial_speed = initial_speed
        
        self.grid = ContinuousSpace(x_max, y_max, torus=True)
        self.schedule = RandomActivation(self)
        self.running = True

        # Create agents
        for i in range(self.num_agents):
            is_sick = self.random.random() < initial_infection_p
            a = EpidemicAgent(i, self, state=State.INFECTED if is_sick else State.UNAFFECTED)
            self.schedule.add(a)
            
            x = self.random.uniform(0, self.grid.width)
            y = self.random.uniform(0, self.grid.height)
            
            # TODO 3: Add agent to randomly selected place on the grid
            # TIP: grid has `place_agent` method
            
            
        self.datacollector = DataCollector(
            model_reporters = {
                'alive': compute_alive,
                'infected': compute_infected,
                'dead': compute_dead,
                'recovered': compute_recovered
            },
            agent_reporters = {
                'state': 'state',
                'speed': 'speed',
                'heading': 'heading',
                'time_infected': 'time_infected'
            }
        )
        
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

## Simulation

In [ ]:
params = {
    'N': 100,
    'initial_infection_p': 0.2,
    'infection_p': 0.4,
    'infection_r': 30,
    'infection_time_min': 20,
    'infection_end_p': 0.8,
    'death_p': 0.5,
    'initial_speed': 10
}

### Ex. 4. Running the simulation

In [ ]:
model = # TODO 4: Initialize model with params
for _ in tqdm(range(100)):
    # TODO 4: Run model step

## Collectors

In [ ]:
m_df = model.datacollector.get_model_vars_dataframe()
m_df

In [ ]:
_ = m_df.plot.line(y='infected')

In [ ]:
a_df = model.datacollector.get_agent_vars_dataframe()
a_df

In [ ]:
_ = a_df[a_df.index.get_level_values('AgentID') == 0].plot.line(y=['heading', 'speed'])

## Visualization

In [ ]:
def plot_states_bokeh(model, iteration, total):
    X = model.datacollector.get_model_vars_dataframe()
    source = ColumnDataSource(X)
    colors = Category10[10]
    items = []
    p = figure(plot_width=600, plot_height=400, tools=[], title=f'step: {iteration}', x_range=(0,total))        
    for i, c in enumerate(X.columns):
        line = Line(x='index', y=c, line_color=colors[i], line_width=3, line_alpha=.8, name=c)
        glyph = p.add_glyph(source, line)
        i+=1
        items.append((c, [glyph]))

    p.xaxis.axis_label = 'Step'
    p.add_layout(Legend(location='center_right', items=items))
    p.background_fill_color = "#ffffff"
    p.background_fill_alpha = 1
    p.legend.label_text_font_size = "10pt"
    p.title.text_font_size = "15pt"
    p.toolbar.logo = None
    p.sizing_mode = 'scale_height'    
    return p

In [ ]:
def plot_agents_bokeh(model):
    def plot_agents(p, agents, color):
        xx = list(map(lambda x: x.pos[0], agents))
        yy = list(map(lambda x: x.pos[1], agents))
        ss = list(map(lambda x: x.infection_radius, agents))
        p.circle_dot(x=xx, y=yy, size=ss, alpha=0.5, fill_color=color)
    
    agents = model.schedule.agents
    w = model.grid.width
    h = model.grid.height
    p = figure(plot_width=500, plot_height=500, x_range=(0, w), y_range=(0, h))
    
    plot_agents(p, list(filter(lambda x: x.state == State.INFECTED, agents)), "red")
    plot_agents(p, list(filter(lambda x: x.state == State.UNAFFECTED, agents)), "blue")
    plot_agents(p, list(filter(lambda x: x.state == State.RECOVERED, agents)), "green")
    
    p.grid.grid_line_color = None    
    p.axis.axis_line_color = None
    p.toolbar.logo = None
    return p

In [ ]:
plot_pane = pn.pane.Bokeh()
grid_pane = pn.pane.Bokeh()
pn.Row(plot_pane, grid_pane, sizing_mode='stretch_width')

## Run simulation

In [ ]:
model = EpidemicModel(
    N=100,
    initial_infection_p=.2,
    infection_p=.5,
    infection_r=30,
    infection_time_min=30,
    infection_end_p=0.7,
    death_p=0.6,
    initial_speed=10
)

TOTAL = 200
for i in range(TOTAL):
    model.step()
    plot_pane.object = plot_states_bokeh(model, i, TOTAL)
    grid_pane.object = plot_agents_bokeh(model)

### Ex. 5. Add metrics
- Implement _infected_ metric computation (`compute_infected` method)
- Rerun simulation to see if it works

### Ex. 6. Add death & recovery
- Implement (`try_die_or_recover` method) agent's recovery or death after some number of iterations (use `time_infected` parameter)
- Rerun simulation to see if it works

# Batch running

In [ ]:
fixed_params = {
    'N': 100,
    'infection_p': 0.5,
    'infection_r': 30,
    'infection_time_min': 30,
    'infection_end_p': 0.7,
    'initial_speed': 10
}

variable_params = {
    'initial_infection_p': np.arange(0.1, 1, 0.1),
    'death_p': np.arange(0.1, 1, 0.1)
}

batch_run = BatchRunner(
    EpidemicModel,
    variable_params,
    fixed_params,
    iterations=3,
    max_steps=100,
    model_reporters={
        'alive': compute_alive,
        'infected': compute_infected,
        'dead': compute_dead,
        'recovered': compute_recovered
    }
)

batch_run.run_all()

In [ ]:
vars_df = batch_run.get_model_vars_dataframe()
vars_df

In [ ]:
input_vars=variable_params.keys()
output_vars=['alive', 'dead', 'recovered', 'infected']

fig, ax = plt.subplots(len(input_vars), len(output_vars), figsize=(18, 10))
fig.subplots_adjust(wspace=0.3)
cmap = plt.get_cmap('tab10')
colors = [rgb2hex(cmap(c)) for c in range(cmap.N)]
for i, o in itertools.product(enumerate(input_vars), enumerate(output_vars)):
    i_idx, i_val = i
    o_idx, o_val = o
    vars_df.plot.scatter(x=i_val, y=o_val, ax=ax[i_idx, o_idx], c=colors[o_idx])